<a href="https://colab.research.google.com/github/Shanmukh459/MusicGenerationUsingLSTM/blob/main/MusicGenerationUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy music21 tensorflow

In [ ]:
#importing required libraries
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#Readind music files
def read_files(file):
  notes = []
  notes_to_parse = None

  #parse midi file
  midi = converter.parse(file)

  #seperating all instruments from the file
  instru = instrument.partitionByInstrument(midi)

  for part in instru.parts:
    if 'Piano' in str(part):
      notes_to_parse = part.recurse()

      #iterate over the part and check if it is note or chord
      #if it is chord split them into notes
      for element in notes_to_parse:
        if type(element) == note.Note:
          notes.append(str(element.pitch))
        elif type(element) == chord.Chord:
          notes.append('.'.join(str(n) for n in element.normalOrder))
  #returning list of notes
  return notes


file_path = ["schubert"]
all_files = glob.glob("/content/drive/MyDrive/schubert/*.mid", recursive=True)


#reading each midi file
notes_array = np.array([read_files(i) for i in tqdm(all_files, position = 0, leave = True)])



  0%|          | 0/29 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2004 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
  7%|▋         | 2/29 [00:23<05:24, 12.00s/it]/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2007 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
 17%|█▋        | 5/29 [00:51<03:58,  9.96s/it]/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2002 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
 21%|██       

In [ ]:
#unique notes
notess = sum(notes_array, [])
unique_notes = list(set(notess))
print("Unique notes: ", len(unique_notes))

#notes along with their frequency

freq = dict(map(lambda x: (x, notess.count(x)), unique_notes))

#threshold frequency
for i in range(30, 100, 20):
  print(i, ":", len(list(filter(lambda x:x[1] >= i, freq.items()))))

Unique notes:  345
30 : 204
50 : 182
70 : 163
90 : 153


In [ ]:
#Changing the threshold frequency to 50. and also adjusting notes_array to only have notes with frequency greater than 50
freq_notes = dict(filter(lambda x: x[1] >= 50, freq.items()))

new_notes = [[i for i in j if i in freq_notes] for j in notes_array]

In [ ]:
#dictionary having key as notes index and value as note
ind2note = dict(enumerate(freq_notes))

#dictionary having note as key and index as value
note2ind = dict(map(reversed, ind2note.items()))

In [ ]:
#timestep
timesteps = 50

#store input and output values
x = [] ; y = []

for i in new_notes:
  for j in range(0, len(i)-timesteps):
    inp = i[j:j+timesteps]
    out = i[j+timesteps]


    x.append(list(map(lambda x: note2ind[x], inp)))
    y.append(note2ind[out])

x_new = np.array(x)
y_new = np.array(y)

In [ ]:
#training and testing
x_new = np.reshape(x_new, (len(x_new), timesteps, 1))
y_new = np.reshape(y_new, (-1, 1))

#splitting the data
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=0.2, random_state=42)

In [ ]:
#Building the model

model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(x_new.shape[1], x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(len(note2ind), activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 182)               46774     
                                                                 
Total params: 902070 (3.44 MB)
Trainable params: 902070 

In [ ]:
#compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#training the model
model.fit(x_train, y_train, batch_size=128, epochs=40, validation_data=(x_test, y_test))

Epoch 1/40
489/489 [==============================] - 491s 994ms/step - loss: 4.7446 - accuracy: 0.0291 - val_loss: 4.6339 - val_accuracy: 0.0421
Epoch 2/40
489/489 [==============================] - 483s 987ms/step - loss: 4.5590 - accuracy: 0.0496 - val_loss: 4.4886 - val_accuracy: 0.0576
Epoch 3/40
489/489 [==============================] - 481s 984ms/step - loss: 4.3711 - accuracy: 0.0715 - val_loss: 4.2606 - val_accuracy: 0.0894
Epoch 4/40
489/489 [==============================] - 483s 988ms/step - loss: 4.1055 - accuracy: 0.1056 - val_loss: 4.0100 - val_accuracy: 0.1195
Epoch 5/40
489/489 [==============================] - 481s 982ms/step - loss: 3.8252 - accuracy: 0.1426 - val_loss: 3.8044 - val_accuracy: 0.1490
Epoch 6/40
489/489 [==============================] - 487s 996ms/step - loss: 3.5716 - accuracy: 0.1719 - val_loss: 3.5892 - val_accuracy: 0.1760
Epoch 7/40
489/489 [==============================] - 494s 1s/step - loss: 3.3550 - accuracy: 0.2055 - val_loss: 3.4758 - va

In [ ]:
#saving the model
model.save("lstm")

In [ ]:
#loading the model
model = load_model("lstm")

#generating random index
index = np.random.randint(0, len(x_test)-1)

#get data present in the generated index place in x_test
music_pattern = x_test[index]
out_pred = []

#iterating till 200 note is generated
for i in range(200):
  music_pattern = music_pattern.reshape(1, len(music_pattern), 1)

  #getting the max prob value from the generated outputs
  pred_index = np.argmax(model.predict(music_pattern))

  #get the note using the preddicted index
  #append the note to the predicted list
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern, pred_index)

  #update the music pattern with one timestamp ahead
  music_pattern = music_pattern[1:]

1/1 [==============================] - 0s 40ms/step


In [ ]:
#saving the file
output_notes = []

for offset, pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []

    for current_note in notes_in_chord:
      i_curr_note = int(current_note)

      new_note = note.Note(i_curr_note)
      new_note.storedInstrument = instrument.Piano()
      notes.append(new_note)

    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)

  else:
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#saving the midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='pred_music.mid')

'pred_music.mid'